Here we calculate the matrix $A$ for the imaging model $y = Ax$. Each element of the matrix $A$ is
$$ a_{ij} = k_i b_j t_j^{ex}(\lambda_i)\int_{\lambda_{i,min}^{em}}^{\lambda_{i,max}^{em}} t_j^{ex}(\lambda) d\lambda$$
We begin by fixing an $i$ and $j$ to calculate one specific element.